In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from ansim_dataset import ansimDataset, create_circular_mask
# from convolution_lstm import encoderConvLSTM, decoderConvLSTM
from ConvLSTM import ConvLSTM
import random
import math
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import os

In [2]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [3]:
img_path = '/home/rliu/ansim/data/data/JPEGImages/'
img_list_csv = '/home/rliu/github/ansim/img_list.csv'
train_csv = '/home/rliu/github/ansim/train.csv'
test_csv = '/home/rliu/github/ansim/test.csv'
output_path = '/home/rliu/ansim/models/very_first.pt'

In [4]:
mask = create_circular_mask(128,128)
trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
trainloader = torch.utils.data.DataLoader(trainset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
testloader = torch.utils.data.DataLoader(testset,
                                             batch_size=8, shuffle=True,
                                             num_workers=4)

In [5]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GPU in use


In [6]:
# def train_model(encoder, decoder, criterion, optimizer, scheduler, num_epochs=25):
#     since = time.time()

#     best_encoder_wts = encoder.state_dict()
#     best_decoder_wts = decoder.state_dict()
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training phase
#         scheduler.step()
#         encoder.train(True)  # Set model to training mode
#         decoder.train(True)  # Set model to training mode
#         running_loss = 0.0
#         running_corrects = 0

#         # Iterate over data.
#         trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
#         trainloader = torch.utils.data.DataLoader(trainset,
#                                                      batch_size=1, shuffle=True,
#                                                      num_workers=4)

#         print("trainloader ready!")
#         testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=20, random_rotate = True, mask = mask, transform=None)
#         testloader = torch.utils.data.DataLoader(testset,
#                                                      batch_size=1, shuffle=True,
#                                                      num_workers=4)
#         print("testloader ready!")
        
#         for data in trainloader:
#             # get the inputs
#             data_split = torch.split(data, 10, dim=1)
#             inputs = data_split[0]
#             target = data_split[1]
# #            print(inputs)
#             # wrap them in Variable
#             if use_gpu:
# #                inputs = Variable(inputs.cuda())
# #                labels = Variable(labels.cuda())
# #                inputs = torch.nn.DataParallel(inputs, device_ids=[0, 1]).cuda()
# #                labels = torch.nn.DataParallel(labels, device_ids=[0, 1]).cuda()
#                 inputs, target = inputs.to(device), target.to(device)
# #                print(inputs)
#             else:
#                 inputs, target = Variable(inputs), Variable(target)

#             # zero the parameter gradients
#             optimizer.zero_grad()
            
#             output, h, c, states = encoder(inputs)
#             output_last = output[0][0].double()
#             h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#             c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#             states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#             x = decoder.activateConv(states_cat)
#             input_d = [x, states]
#             output_d, h_d, c_d, states_d = decoder(input_d)
#             predicted = torch.cat(output_d, dim=0, out=None).double()
            
            
#             loss = criterion(predicted, target)
#             # forward
# #            outputs = model(inputs)
# #            _, preds = torch.max(outputs.data, 1)
# #            loss = criterion(outputs, labels)

#             loss.backward()
#             optimizer.step()

#             # statistics
#             iter_loss = loss.item()
#             running_loss += loss.item()    
#             epoch_loss = running_loss / len(trainset)
            
#             print('{} Loss: {:.4f} batch_loss: {:d}'.format(
#                 "train", epoch_loss, iter_loss))
        
#         with torch.no_grad():
#             for data in testloader:
#                 data_split = torch.split(data, 10, dim=1)
#                 inputs = data_split[0]
#                 target = data_split[1]
                
#                 if use_gpu:
#                     inputs, target = inputs.to(device), target.to(device)
#                 else:
#                     inputs, target = Variable(inputs), Variable(target)

                    
#                 output, h, c, states = encoder(inputs)
#                 output_last = output[0][0].double()
#                 h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#                 c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#                 states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#                 x = decoder.activateConv(states_cat)
#                 input_d = [x, states]
#                 output_d, h_d, c_d, states_d = decoder(input_d)
#                 predicted = torch.cat(output_d, dim=0, out=None).double()
            
#                 loss_test = criterion(predicted, target)
#                 iter_loss_test = loss_test.item()
#                 running_loss_test += loss_test.item()    
#                 epoch_loss_test = running_loss_test / len(testset)

#         print('Loss on the test images: %.5f %%' % (
#             epoch_loss_test))
        

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     encoder.load_state_dict(best_encoder_wts)
#     decoder.load_state_dict(best_decoder_wts)
#     return encoder, decoder

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, batch_size = 4, step_size = 20):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training phase
        scheduler.step()
        model.train(True)  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        

        # Iterate over data.
        trainset = ansimDataset(img_list_csv = img_list_csv, seq_csv = train_csv, root_dir = img_path, step=step_size, random_rotate = True, mask = mask, transform=None)
        trainloader = torch.utils.data.DataLoader(trainset,
                                                     batch_size=batch_size, shuffle=True,
                                                     num_workers=4)

        print("trainloader ready!")
        testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=step_size, random_rotate = True, mask = mask, transform=None)
        testloader = torch.utils.data.DataLoader(testset,
                                                     batch_size=batch_size, shuffle=True,
                                                     num_workers=4)
        print("testloader ready!")
        
        for data in trainloader:
            # get the inputs
            data_split = torch.split(data, int(data.shape[1]/2), dim=1)
            inputs = data_split[0]
            target = data_split[1]
#            print(inputs)
            # wrap them in Variable
            if use_gpu:
                inputs, target = inputs.to(device), target.to(device)
            else:
                inputs, target = Variable(inputs), Variable(target)

            # zero the parameter gradients
            optimizer.zero_grad()
            
#             output, h, c, states = encoder(inputs)
#             output_last = output[0][0].double()
#             h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#             c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#             states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#             x = decoder.activateConv(states_cat)
#             input_d = [x, states]
#             output_d, h_d, c_d, states_d = decoder(input_d)
#             predicted = torch.cat(output_d, dim=0, out=None).double()
            
            _, _, predicted = model(inputs)
            
            m = nn.Sigmoid()
            loss = criterion(m(predicted), m(target))
            # forward
#            outputs = model(inputs)
#            _, preds = torch.max(outputs.data, 1)
#            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # statistics
            iter_loss = loss.item()
            running_loss += loss.item()    
            epoch_loss = running_loss / len(trainset)
            
            print('{} Loss: {:.4f} batch_loss: {:f}'.format(
                "train", epoch_loss, iter_loss))
        
        with torch.no_grad():
            running_loss_test = 0.0
            for data in testloader:
                data_split = torch.split(data, int(data.shape[1]/2), dim=1)
                inputs = data_split[0]
                target = data_split[1]
                
                
                if use_gpu:
                    inputs, target = inputs.to(device), target.to(device)
                else:
                    inputs, target = Variable(inputs), Variable(target)

                    
#                 output, h, c, states = encoder(inputs)
#                 output_last = output[0][0].double()
#                 h1,h2,h3,h4,h5 = states[0][0],states[1][0],states[2][0],states[3][0],states[4][0]
#                 c1,c2,c3,c4,c5 = states[0][1],states[1][1],states[2][1],states[3][1],states[4][1]
#                 states_cat = torch.cat((h1,h2,h3,h4,h5,c1,c2,c3,c4,c5), dim=1, out=None)
#                 x = decoder.activateConv(states_cat)
#                 input_d = [x, states]
#                 output_d, h_d, c_d, states_d = decoder(input_d)
#                 predicted = torch.cat(output_d, dim=0, out=None).double()

                _, _, predicted = model(inputs)
                
                m = nn.Sigmoid()
                loss_test = criterion(m(predicted), m(target))
                
                iter_loss_test = loss_test.item()
                running_loss_test += loss_test.item()    
                epoch_loss_test = running_loss_test / len(testset)

        print('Loss on the test images: %.5f ' % (
            epoch_loss_test))
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return encoder, decoder

In [ ]:
# transfer learning resnet18
step_size = 20
model = ConvLSTM(input_size=(128,128),
                 input_dim=1,
                 hidden_dim=[64, 64, 128],
                 kernel_size=(3, 3),
                 num_layers=3,
                 predict_steps=int(step_size/2),
                 batch_first=True,
                 bias=True,
                 return_all_layers=True).cuda()

if use_gpu:
#     encoder = torch.nn.DataParallel(encoder)
#     decoder = torch.nn.DataParallel(decoder)
    model = torch.nn.DataParallel(model)
    model.to(device)

#criterion = nn.MSELoss()
criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

# train model
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, batch_size = 1, step_size = 20, num_epochs=10)
torch.save(model, output_path)

Epoch 0/9
----------
trainloader ready!
testloader ready!
train Loss: 0.0009 batch_loss: 0.696245
train Loss: 0.0016 batch_loss: 0.552550
train Loss: 0.0019 batch_loss: 0.247794
train Loss: 0.0022 batch_loss: 0.212266
train Loss: 0.0026 batch_loss: 0.259702
train Loss: 0.0029 batch_loss: 0.253041
train Loss: 0.0032 batch_loss: 0.230204
train Loss: 0.0035 batch_loss: 0.216968
train Loss: 0.0037 batch_loss: 0.189752
train Loss: 0.0040 batch_loss: 0.188197
train Loss: 0.0042 batch_loss: 0.189403
train Loss: 0.0044 batch_loss: 0.179107
train Loss: 0.0047 batch_loss: 0.206736
train Loss: 0.0049 batch_loss: 0.173229
train Loss: 0.0052 batch_loss: 0.170160
train Loss: 0.0054 batch_loss: 0.164219
train Loss: 0.0056 batch_loss: 0.162365
train Loss: 0.0058 batch_loss: 0.162835
train Loss: 0.0060 batch_loss: 0.174340
train Loss: 0.0062 batch_loss: 0.162468
train Loss: 0.0065 batch_loss: 0.163850
train Loss: 0.0067 batch_loss: 0.158713
train Loss: 0.0069 batch_loss: 0.163574
train Loss: 0.0071 bat

train Loss: 0.0431 batch_loss: 0.150875
train Loss: 0.0433 batch_loss: 0.150282
train Loss: 0.0435 batch_loss: 0.150977
train Loss: 0.0437 batch_loss: 0.150298
train Loss: 0.0439 batch_loss: 0.151443
train Loss: 0.0441 batch_loss: 0.153189
train Loss: 0.0443 batch_loss: 0.151163
train Loss: 0.0445 batch_loss: 0.150218
train Loss: 0.0447 batch_loss: 0.156702
train Loss: 0.0449 batch_loss: 0.150374
train Loss: 0.0451 batch_loss: 0.150142
train Loss: 0.0453 batch_loss: 0.150222
train Loss: 0.0455 batch_loss: 0.150464
train Loss: 0.0457 batch_loss: 0.150627
train Loss: 0.0458 batch_loss: 0.150109
train Loss: 0.0460 batch_loss: 0.150363
train Loss: 0.0462 batch_loss: 0.149866
train Loss: 0.0464 batch_loss: 0.150321
train Loss: 0.0466 batch_loss: 0.151026
train Loss: 0.0468 batch_loss: 0.149754
train Loss: 0.0470 batch_loss: 0.150032
train Loss: 0.0472 batch_loss: 0.151566
train Loss: 0.0474 batch_loss: 0.150951
train Loss: 0.0476 batch_loss: 0.151447
train Loss: 0.0478 batch_loss: 0.156310


train Loss: 0.0834 batch_loss: 0.150957
train Loss: 0.0836 batch_loss: 0.150202
train Loss: 0.0838 batch_loss: 0.151360
train Loss: 0.0840 batch_loss: 0.154002
train Loss: 0.0842 batch_loss: 0.150977
train Loss: 0.0844 batch_loss: 0.150226
train Loss: 0.0846 batch_loss: 0.154952
train Loss: 0.0848 batch_loss: 0.150037
train Loss: 0.0850 batch_loss: 0.150421
train Loss: 0.0852 batch_loss: 0.150418
train Loss: 0.0854 batch_loss: 0.150104
train Loss: 0.0856 batch_loss: 0.150060
train Loss: 0.0858 batch_loss: 0.151109
train Loss: 0.0860 batch_loss: 0.150110
train Loss: 0.0862 batch_loss: 0.149998
train Loss: 0.0864 batch_loss: 0.155535
train Loss: 0.0866 batch_loss: 0.149973
train Loss: 0.0868 batch_loss: 0.150185
train Loss: 0.0870 batch_loss: 0.150025
train Loss: 0.0872 batch_loss: 0.149984
train Loss: 0.0874 batch_loss: 0.150120
train Loss: 0.0876 batch_loss: 0.150010
train Loss: 0.0878 batch_loss: 0.150162
train Loss: 0.0880 batch_loss: 0.150275
train Loss: 0.0882 batch_loss: 0.151233


train Loss: 0.1238 batch_loss: 0.150051
train Loss: 0.1240 batch_loss: 0.154558
train Loss: 0.1242 batch_loss: 0.150024
train Loss: 0.1244 batch_loss: 0.151297
train Loss: 0.1246 batch_loss: 0.150761
train Loss: 0.1248 batch_loss: 0.151039
train Loss: 0.1250 batch_loss: 0.149918
train Loss: 0.1252 batch_loss: 0.150186
train Loss: 0.1254 batch_loss: 0.153952
train Loss: 0.1255 batch_loss: 0.150044
train Loss: 0.1257 batch_loss: 0.150384
train Loss: 0.1259 batch_loss: 0.150096
train Loss: 0.1261 batch_loss: 0.151371
train Loss: 0.1263 batch_loss: 0.150248
train Loss: 0.1265 batch_loss: 0.149977
train Loss: 0.1267 batch_loss: 0.150041
train Loss: 0.1269 batch_loss: 0.149983
train Loss: 0.1271 batch_loss: 0.151206
train Loss: 0.1273 batch_loss: 0.151093
train Loss: 0.1275 batch_loss: 0.149951
train Loss: 0.1277 batch_loss: 0.149959
train Loss: 0.1279 batch_loss: 0.149970
train Loss: 0.1281 batch_loss: 0.150042
train Loss: 0.1283 batch_loss: 0.150255
train Loss: 0.1285 batch_loss: 0.149889


train Loss: 0.0098 batch_loss: 0.150306
train Loss: 0.0100 batch_loss: 0.150802
train Loss: 0.0102 batch_loss: 0.149968
train Loss: 0.0104 batch_loss: 0.149927
train Loss: 0.0106 batch_loss: 0.149935
train Loss: 0.0108 batch_loss: 0.150010
train Loss: 0.0110 batch_loss: 0.155453
train Loss: 0.0112 batch_loss: 0.150482
train Loss: 0.0114 batch_loss: 0.150090
train Loss: 0.0116 batch_loss: 0.150017
train Loss: 0.0118 batch_loss: 0.151357
train Loss: 0.0120 batch_loss: 0.149960
train Loss: 0.0122 batch_loss: 0.150991
train Loss: 0.0124 batch_loss: 0.150015
train Loss: 0.0126 batch_loss: 0.150010
train Loss: 0.0128 batch_loss: 0.150825
train Loss: 0.0130 batch_loss: 0.150031
train Loss: 0.0132 batch_loss: 0.150001
train Loss: 0.0134 batch_loss: 0.155754
train Loss: 0.0136 batch_loss: 0.150080
train Loss: 0.0138 batch_loss: 0.151737
train Loss: 0.0140 batch_loss: 0.155264
train Loss: 0.0142 batch_loss: 0.150803
train Loss: 0.0144 batch_loss: 0.149681
train Loss: 0.0146 batch_loss: 0.149580


train Loss: 0.0501 batch_loss: 0.150098
train Loss: 0.0503 batch_loss: 0.151010
train Loss: 0.0505 batch_loss: 0.152922
train Loss: 0.0507 batch_loss: 0.150073
train Loss: 0.0509 batch_loss: 0.150026
train Loss: 0.0511 batch_loss: 0.150379
train Loss: 0.0513 batch_loss: 0.151250
train Loss: 0.0515 batch_loss: 0.150747
train Loss: 0.0517 batch_loss: 0.150179
train Loss: 0.0519 batch_loss: 0.150166
train Loss: 0.0521 batch_loss: 0.151769
train Loss: 0.0523 batch_loss: 0.150041
train Loss: 0.0525 batch_loss: 0.150087
train Loss: 0.0527 batch_loss: 0.150092
train Loss: 0.0529 batch_loss: 0.150294
train Loss: 0.0531 batch_loss: 0.150923
train Loss: 0.0533 batch_loss: 0.151753
train Loss: 0.0535 batch_loss: 0.150026
train Loss: 0.0536 batch_loss: 0.149899
train Loss: 0.0538 batch_loss: 0.149900
train Loss: 0.0540 batch_loss: 0.154273
train Loss: 0.0542 batch_loss: 0.149977
train Loss: 0.0544 batch_loss: 0.151634
train Loss: 0.0546 batch_loss: 0.149956
train Loss: 0.0548 batch_loss: 0.155854


train Loss: 0.0904 batch_loss: 0.150912
train Loss: 0.0906 batch_loss: 0.153149
train Loss: 0.0908 batch_loss: 0.150849
train Loss: 0.0910 batch_loss: 0.150335
train Loss: 0.0912 batch_loss: 0.155538
train Loss: 0.0914 batch_loss: 0.150034
train Loss: 0.0916 batch_loss: 0.151068
train Loss: 0.0918 batch_loss: 0.150058
train Loss: 0.0920 batch_loss: 0.150357
train Loss: 0.0922 batch_loss: 0.150018
train Loss: 0.0924 batch_loss: 0.151357
train Loss: 0.0926 batch_loss: 0.150050
train Loss: 0.0928 batch_loss: 0.150040
train Loss: 0.0930 batch_loss: 0.150127
train Loss: 0.0932 batch_loss: 0.150783
train Loss: 0.0934 batch_loss: 0.150604
train Loss: 0.0936 batch_loss: 0.150012
train Loss: 0.0938 batch_loss: 0.151662
train Loss: 0.0939 batch_loss: 0.149883
train Loss: 0.0941 batch_loss: 0.149929
train Loss: 0.0943 batch_loss: 0.149893
train Loss: 0.0945 batch_loss: 0.152996
train Loss: 0.0947 batch_loss: 0.150100
train Loss: 0.0949 batch_loss: 0.149968
train Loss: 0.0951 batch_loss: 0.150005


train Loss: 0.1307 batch_loss: 0.150517
train Loss: 0.1309 batch_loss: 0.149954
train Loss: 0.1311 batch_loss: 0.150014
train Loss: 0.1313 batch_loss: 0.153593
train Loss: 0.1315 batch_loss: 0.149803
train Loss: 0.1317 batch_loss: 0.150010
train Loss: 0.1319 batch_loss: 0.150157
train Loss: 0.1321 batch_loss: 0.156021
train Loss: 0.1323 batch_loss: 0.150259
train Loss: 0.1325 batch_loss: 0.150768
train Loss: 0.1327 batch_loss: 0.151055
train Loss: 0.1329 batch_loss: 0.151096
train Loss: 0.1331 batch_loss: 0.149877
train Loss: 0.1333 batch_loss: 0.154673
train Loss: 0.1335 batch_loss: 0.151603
train Loss: 0.1337 batch_loss: 0.149815
train Loss: 0.1339 batch_loss: 0.150129
train Loss: 0.1341 batch_loss: 0.150250
train Loss: 0.1343 batch_loss: 0.150059
train Loss: 0.1345 batch_loss: 0.152536
train Loss: 0.1346 batch_loss: 0.150293
train Loss: 0.1348 batch_loss: 0.150127
train Loss: 0.1350 batch_loss: 0.150058
train Loss: 0.1352 batch_loss: 0.150070
train Loss: 0.1354 batch_loss: 0.150225


train Loss: 0.0197 batch_loss: 0.150098
train Loss: 0.0199 batch_loss: 0.151773
train Loss: 0.0200 batch_loss: 0.150782
train Loss: 0.0202 batch_loss: 0.149864
train Loss: 0.0204 batch_loss: 0.149896
train Loss: 0.0206 batch_loss: 0.150077
train Loss: 0.0208 batch_loss: 0.150031
train Loss: 0.0210 batch_loss: 0.151396
train Loss: 0.0212 batch_loss: 0.152875
train Loss: 0.0214 batch_loss: 0.150722
train Loss: 0.0216 batch_loss: 0.150154
train Loss: 0.0218 batch_loss: 0.153878
train Loss: 0.0220 batch_loss: 0.151893
train Loss: 0.0222 batch_loss: 0.150506
train Loss: 0.0224 batch_loss: 0.149995
train Loss: 0.0226 batch_loss: 0.149976
train Loss: 0.0228 batch_loss: 0.150794
train Loss: 0.0230 batch_loss: 0.152571
train Loss: 0.0232 batch_loss: 0.149923
train Loss: 0.0234 batch_loss: 0.149922
train Loss: 0.0236 batch_loss: 0.150273
train Loss: 0.0238 batch_loss: 0.149887
train Loss: 0.0240 batch_loss: 0.150768
train Loss: 0.0242 batch_loss: 0.151448
train Loss: 0.0244 batch_loss: 0.150568


train Loss: 0.0599 batch_loss: 0.149976
train Loss: 0.0601 batch_loss: 0.150095
train Loss: 0.0603 batch_loss: 0.151993
train Loss: 0.0605 batch_loss: 0.150635
train Loss: 0.0607 batch_loss: 0.153561
train Loss: 0.0609 batch_loss: 0.150160
train Loss: 0.0611 batch_loss: 0.150203
train Loss: 0.0613 batch_loss: 0.150001
train Loss: 0.0615 batch_loss: 0.149978
train Loss: 0.0617 batch_loss: 0.151709
train Loss: 0.0619 batch_loss: 0.149886
train Loss: 0.0621 batch_loss: 0.154005
train Loss: 0.0623 batch_loss: 0.155077
train Loss: 0.0625 batch_loss: 0.150785
train Loss: 0.0627 batch_loss: 0.151157
train Loss: 0.0629 batch_loss: 0.153665
train Loss: 0.0631 batch_loss: 0.151116
train Loss: 0.0633 batch_loss: 0.150326
train Loss: 0.0635 batch_loss: 0.154835
train Loss: 0.0637 batch_loss: 0.149979
train Loss: 0.0639 batch_loss: 0.151641
train Loss: 0.0641 batch_loss: 0.150905
train Loss: 0.0643 batch_loss: 0.149988
train Loss: 0.0645 batch_loss: 0.152228
train Loss: 0.0647 batch_loss: 0.149946


In [ ]:
# testset = ansimDataset(img_list_csv = img_list_csv, seq_csv = test_csv, root_dir = img_path, step=step_size, random_rotate = True, mask = mask, transform=None)
# testloader = torch.utils.data.DataLoader(testset,
#                                              batch_size=1, shuffle=True,
#                                              num_workers=1)
# dataiter = iter(testloader)
# data = dataiter.next()
# data_split = torch.split(data, int(data.shape[1]/2), dim=1)
# inputs = data_split[0]
# target = data_split[1]
# outputs = model(inputs)


In [ ]:
count_parameters(model)

In [10]:
m = nn.Sigmoid()
input = Variable(torch.randn(3, 1, 512, 64, 32)).cuda()
a = m(input)

In [11]:
a.shape

torch.Size([3, 1, 512, 64, 32])